In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
import methods
import pickle
import warnings
import plotly as py

warnings.filterwarnings('ignore')

In [ ]:
% matplotlib inline

## Getting Jaspers raw data

In [ ]:
clip_name = '20180404_183056.mp4'
# image_h, image_w, fps = methods.determine_video_meta_data('C:/Users/jaspe/tf-openpose/clips/{}'.format(clip_name))
people_per_file = methods.get_openpose_output('coordinates/{}'.format(clip_name))

In [ ]:
image_h = 1080
image_w = 1920
fps = 60

In [ ]:
clip_stats = methods.get_clip_stats(clip_name)
if clip_stats != None:
    length, weight, distance = clip_stats

## Load prepared data

In [ ]:
# plottables_per_file, period_person_division = methods.\
#             get_plottables_per_file_and_period_person_division(people_per_file, fps, connections)

period_person_division = methods.get_period_person_division(people_per_file, fps)    
#plottables_per_file = methods.get_plottables_per_file(people_per_file, connections)
    
    
person_period_division = methods.get_person_period_division(period_person_division)

mean_x_per_person = methods.get_mean_x_per_person(person_period_division)

normalized_moved_distance_per_person = methods.normalize_moved_distance_per_person(mean_x_per_person)

# Only include identified people that move more than a set movement threshold
maximum_normalized_distance = max(normalized_moved_distance_per_person.values())
movement_threshold = maximum_normalized_distance / 4
moving_people = [key for key, value in normalized_moved_distance_per_person.items() if value > movement_threshold]

person_plottables_df = methods.get_person_plottables_df(mean_x_per_person, moving_people)

dbscan_subsets = methods.get_dbscan_subsets(maximum_normalized_distance, person_plottables_df)

# mean_x_per_moving_person = {key:np.array([[period,x] for period,x in value.items()]) 
#                             for key,value in mean_x_per_person.items() if key in moving_people}

# links = methods.get_links(moving_people, mean_x_per_moving_person)

# linked_people = methods.get_linked_people(maximum_normalized_distance, links)

# plottable_subsets = dbscan_subsets + linked_people

# all_moving_people = set(chain.from_iterable(plottable_subsets))

# plottable_people = plottable_subsets[
#     np.argmax([sum([len(person_period_division[person]) for person in subset]) for subset in plottable_subsets])]

max_dbscan_subset = dbscan_subsets[
    np.argmax([sum([len(person_period_division[person]) for person in subset]) for subset in dbscan_subsets])]

plottable_people = methods.determine_plottable_people(person_plottables_df, 
                                                      max_dbscan_subset, 
                                                      maximum_normalized_distance*4,
                                                      maximum_normalized_distance**2)

running_fragments, turning_fragments, fragments = \
            methods.get_running_and_turning_fragments(plottable_people, mean_x_per_person, person_plottables_df, moving_people,
                                                     fps)
    
coord_df = methods.get_dataframe_from_coords(methods.prepare_data_for_plotting(period_person_division, 
                                                                           plottable_people, 
                                                                           running_fragments))

rotation_angle = methods.get_rotation_angle(coord_df)

coord_df = methods.process_coord_df(coord_df)

feature_df = methods.to_feature_df(coord_df, 1)

period_person_division = {period:{person: np.array([methods.rotate((x,y), rotation_angle)+(z,) for x,y,z in coords])
                                  for person, coords in period_dictionary.items()}
                                  for period, period_dictionary in period_person_division.items()}

person_plottables, running_plottables, turning_plottables = \
                methods.get_plottables(period_person_division, plottable_people, running_fragments, turning_fragments)

# Testing variation of joints

In [ ]:
test_df = coord_df[coord_df['Point'] == 'Left Ear']

In [ ]:
points = list(set(point for fragment in fragments for point in fragment))
running_points = list(set(point for fragment in running_fragments for point in fragment))

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(x=test_df['Frame'], y=test_df['y'])
for point in points:
    plt.axvline(point)
for point in running_points:
    plt.axvline(point, c='r')

# Feature df

In [ ]:
feature_df

# Plotting

In [ ]:
% matplotlib notebook

In [ ]:
methods.plot_person(running_plottables, image_h, image_w, zoom=True, pad=3, sleep=0.05)

In [ ]:
# coord_list = methods.prepare_data_for_plotting(period_person_division, plottable_people, running_fragments)
# coord_df = methods.get_dataframe_from_coords(coord_list)
# feature_df, coord_df = methods.to_feature_df(coord_df)
# feature_df = methods.forward_leaning(feature_df, coord_df)
# feature_df.head()